In [1]:
from utils import prepare_data
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from tqdm.notebook import tnrange, tqdm_notebook

In [2]:
q, a, pairs, vector = prepare_data('test', 'glove.42B.300d/glove.42B.300d.txt', small=True)

Reading test -------
Read 4041 sentence pairs
Counting words
Counted words:
In questions: 5087 words
In answers: 394 words


In [5]:
def epoch_train(model, optimizer, batch_size, pairs)
    
    # Set the model in train mode
    model.train()
    
    # Gets number total number of rows for training
    n_records = len(pairs)
    
    # Shuffle the row indexes 
    perm = np.random.permutation(n_records)
    
    st = 0
    
    while st < n_records:
        
        ed = st + batch_size if (st + batch_size) < n_records else n_records
    
        encoder_in, decoder_in = to_batch_sequence(pairs, st, ed, perm)

        # Calculate outputs and loss
        output_values, loss = model(encoder_in, decoder_in)
        
        # Clear gradients (pytorch accumulates gradients by default)
        optimizer.zero_grad() 

        # Backpropagation & weight adjustment
        loss.backward()
        optimizer.step()
        
        st = ed

print(f"Optimization ended successfully")

SyntaxError: invalid syntax (<ipython-input-5-e9e706937ea8>, line 1)

In [4]:
np.random.permutation(len(pairs))

array([1882, 3441, 3572, ..., 3598,   73, 3906])